Problem Formulation

---



ai = supply i ; 

bj = demand  at station j ; 

dij = distance between plant i and station j (Haversin distance km)

> This is a test and not a real life problem. Because of this I can admit the use of haversine distance (calculated from given  x and y coordinates only for AI95) , BUT this coordinates doesn't take into account the physical road barriers such as  traffic, rivers, lakes, buildings etc. In optimization problems it is better to use the road lenth.


cij = F⋅dij  shipping cost  between plant i  and station j

xij  = amount of petroleum to ship from plant i  to station j 

where xij ≥0 , for all i,j

Observe supply limit at plant i : ∑j xij≤ai for all i

Satisfy demand at station j:∑i xij≥bj for all j  
> **Mimize ∑i ∑j cij*xij**










In [134]:
import pandas as pd
from math import radians, cos, sin, asin, sqrt
from collections import defaultdict
!pip install optlang
from optlang import Variable, Constraint, Objective, Model

df1 = pd.read_csv('/content/NPZ.txt', delimiter="\t")
df2 = pd.read_csv('/content/STATIONS.txt', delimiter="\t")
df=df1.merge(df2, left_on='AI', right_on='AI')
df=df.loc[df['AI'] == 95] 



def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km


for index, row in df.iterrows():
    df.loc[index, 'distance'] = haversine(row['x_x'], row['y_x'], row['x_y'], row['y_y'])

demand=dict(zip(df.Station, df.Demand))
supply=dict(zip(df.NPZ, df.Supply)) 


d = defaultdict(dict)
for x, y, z in zip(df.NPZ, df.Station, df.distance):
    d[x][y] = z

distances = d

TransPrice = 2.2  # TransPrice

# Define variables
variables = {}
for origin in supply:
    variables[origin] = {}
    for destination in demand:
        # Construct a variable with a name, bounds and type
        var = Variable(name="{}_to_{}".format(origin, destination), lb=0, type="integer")
        variables[origin][destination] = var

# Define constraints
constraints = []
for origin in supply:
    const = Constraint(
        sum(variables[origin].values()),
        ub=supply[origin],
        name="{}_supply".format(origin)
    )
    constraints.append(const)
for destination in demand:
    const = Constraint(
        sum(row[destination] for row in variables.values()),
        lb=demand[destination],
        name="{}_demand".format(destination)
    )
    constraints.append(const)

# Define the objective
obj = Objective(
    sum(TransPrice * distances[ori][dest] * variables[ori][dest] for ori in supply for dest in demand),
    direction="min"
)
# We can print the objective and constraints
print(obj)
print("")
for const in constraints:
    print(const)

print("")

# Put everything together in a Model
model = Model()
model.add(constraints)  # Variables are added implicitly
model.objective = obj

# Optimize and print the solution
status = model.optimize()
print("Status:", status)
print("Objective value:", model.objective.value )

print("")
for var in model.variables:
    print(var.name, ":", var.primal )
    

Minimize
5306.45607849077*NPZ_1_to_STAT_1 + 8343.01131271425*NPZ_1_to_STAT_10 + 2213.04201166737*NPZ_1_to_STAT_11 + 13385.3224860272*NPZ_1_to_STAT_12 + 7861.99694743877*NPZ_1_to_STAT_13 + 3286.11302235518*NPZ_1_to_STAT_2 + 13648.6624904094*NPZ_1_to_STAT_3 + 7765.7704066301*NPZ_1_to_STAT_4 + 11364.237788879*NPZ_1_to_STAT_5 + 5079.97569133224*NPZ_1_to_STAT_6 + 9559.19084693836*NPZ_1_to_STAT_7 + 6681.22074949125*NPZ_1_to_STAT_8 + 6656.69121271031*NPZ_1_to_STAT_9 + 17376.4138983981*NPZ_2_to_STAT_1 + 7540.16469787676*NPZ_2_to_STAT_10 + 13450.586067649*NPZ_2_to_STAT_11 + 1952.66132705617*NPZ_2_to_STAT_12 + 15065.857942212*NPZ_2_to_STAT_13 + 12825.163066646*NPZ_2_to_STAT_2 + 13480.5738394507*NPZ_2_to_STAT_3 + 18805.51052132*NPZ_2_to_STAT_4 + 10479.5250985028*NPZ_2_to_STAT_5 + 11347.8973283813*NPZ_2_to_STAT_6 + 7343.90514998867*NPZ_2_to_STAT_7 + 17828.269327214*NPZ_2_to_STAT_8 + 5566.26660978921*NPZ_2_to_STAT_9 + 8223.84976562786*NPZ_3_to_STAT_1 + 4693.684545715*NPZ_3_to_STAT_10 + 7086.1835598